In [ ]:
import os
import cv2 as cv
import torch
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [ ]:
# paths
NOTEBOOK_DIR = os.getcwd()
ROOT = os.path.join(NOTEBOOK_DIR, os.path.pardir)
IMG_DIR = os.path.join(ROOT, "images")
CHECKPOINTS_DIR = os.path.join(ROOT, "checkpoints")
YOLO_CHECKPOINT_PATH = os.path.join(CHECKPOINTS_DIR, "yolo11s-seg.pt")

# torch cfg
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# YOLO cfg
CLASSES_TO_DETECT = list(range(0, 9)) # detects persons and vehichles

In [ ]:
model = YOLO(YOLO_CHECKPOINT_PATH)
model.to(DEVICE)

In [ ]:
imgs = []
for filename in os.listdir(IMG_DIR):
    img_path = os.path.join(IMG_DIR, filename)
    img = cv.imread(img_path)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    imgs.append(img)

preds = model.predict(imgs, classes=CLASSES_TO_DETECT)

In [ ]:
# Plotting
for i, pred in enumerate(preds):
    plt.figure(figsize=(10, 6))
    plt.imshow(imgs[i])

    boxes = pred.boxes.xyxy.cpu().numpy()
    labels = pred.boxes.cls.cpu().numpy()
    scores = pred.boxes.conf.cpu().numpy()

    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        class_name = model.names[int(label)]
        plt.plot([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], 
                 linewidth=2, color="red")
        plt.text(x1, y1 - 5, f"{class_name} {score:.2f}", 
                 fontsize=10, color="red", bbox=dict(facecolor="white", alpha=0.7))

plt.axis("off")
plt.show()